# Bike Rental

**Daily bike rental ridership prediction using an artificial neural network in Keras**

**Supervised Learning. Regression**


Based on the [first neural network project](https://github.com/udacity/deep-learning/tree/master/first-neural-network) from the [Deep Learning Nanodegree Foundation of Udacity](https://www.udacity.com/course/deep-learning-nanodegree-foundation--nd101)

Click [Here](https://github.com/angelmtenor/deep-learning/blob/master/first-neural-network/dlnd-your-first-neural-network.ipynb) to check my original solution in Numpy

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ds_boost
from tensorflow import keras

log = ds_boost.logger.init(level="DEBUG", save_log=False)

ds_boost.set_parent_execution_path()
ds_boost.info_system()
ds_boost.reproducible(seed=0)

N_JOBS = 10

## 1. Data Processing and Exploratory Data Analysis

In [ ]:
data_path = "data/Bike-Sharing-Dataset/hour.csv"
target = ["cnt", "casual", "registered"]

df = pd.read_csv(data_path)

### Explore the data

In [ ]:
ds_boost.info_data(df, target)

In [ ]:
df.head(3)

#### Numerical Data

In [ ]:
df.describe(percentiles=[0.5])

#### Missing values

In [ ]:
ds_boost.missing(df);

### Transform the data

#### Add new features

In [ ]:
ind = pd.to_datetime(df["dteday"])
df["day"] = pd.DatetimeIndex(ind).day

#### Remove irrelevant features

In [ ]:
droplist = ["atemp", "day", "instant"]
df = df.drop(droplist, axis="columns")

#### Classify variables

In [ ]:
numerical = ["temp", "hum", "windspeed", "cnt", "casual", "registered"]

df = ds_boost.sort_columns_by_type(df, target, numerical)

ds_boost.get_types(df)

### Visualize the data

This dataset has the number of riders for each hour of each day from January 1 2011 to December 31 2012. The number of riders is split between casual and registered, summed up in the `cnt` column. We also have information about temperature, humidity, and windspeed, all of these likely affecting the number of riders. Below is a plot showing the hourly rentals over the first 10 days in the data set. The weekends have lower over all ridership and there are spikes when people are biking to and from work during the week.

In [ ]:
df[: 24 * 10].plot(x="dteday", y="cnt");

#### Categorical features

In [ ]:
ds_boost.show_categorical(df[["holiday", "workingday", "weathersit"]], sharey=True)

#### Target vs Categorical features

In [ ]:
ds_boost.show_target_vs_categorical(df.drop(["dteday", "season"], axis="columns"), target, ncols=7)

In [ ]:
g = sns.PairGrid(df, y_vars="casual", x_vars="registered", height=5, aspect=9 / 4, hue="weathersit")
g.map(sns.regplot, fit_reg=False).add_legend()
g.axes[0, 0].set_ylim(0, 350)

This plot shows the differences between the number of registered and casual riders for the different weather situations. Most of the riders are registered with very bad weather (`weathersit=4`).

#### Numerical features

In [ ]:
ds_boost.show_numerical(df, kde=True, ncols=6)

#### Target vs numerical features

In [ ]:
ds_boost.show_target_vs_numerical(df, target, jitter=0.05)

#### Correlation between numerical features and target

In [ ]:
ds_boost.correlation(df, target, figsize=(10, 4))

## 2. Neural Network model

### Select the features

In [ ]:
droplist = ["dteday"]  # features to drop from the model

# For the model 'data' instead of 'df'
data = df.copy()
data = data.drop(droplist, axis="columns")
data.head(3)

### Scale numerical variables

Shift and scale numerical variables to a standard normal distribution. The scaling factors are saved to be used for predictions.

In [ ]:
data, scale_param = ds_boost.scale(data)

### Create dummy features
Replace categorical features (no target) with dummy features

In [ ]:
data, dict_dummies = ds_boost.replace_by_dummies(data, target)
model_features = [f for f in data if f not in target]  # sorted neural network inputs
data.head(3)

### Split the data into training, validation, and test sets
Data leakage: Test set hidden when training the model, but seen when preprocessing the dataset

In [ ]:
# Save the last 21 days as a test set
test = data[-21 * 24 :]
train = data[: -21 * 24]

# Hold out the last 60 days of the remaining data as a validation set
val = train[-60 * 24 :]
train = train[: -60 * 24]

# Separate the data into features(x) and targets(y)
x_train, y_train = train.drop(target, axis=1).values, train[target].values
x_val, y_val = val.drop(target, axis=1).values, val[target].values
x_test, y_test = test.drop(target, axis=1).values, test[target].values

print(f"train size \t X:{x_train.shape} \t Y:{y_train.shape}")
print(f"val size \t X:{x_val.shape} \t Y:{y_val.shape}")
print(f"test size  \t X:{x_test.shape} \t Y:{y_test.shape} ")

# convert to tensors
x_train, y_train, x_val, y_val, x_test, y_test = ds_boost.convert_to_tensors(
    (x_train, y_train, x_val, y_val, x_test, y_test)
)

### Build the Neural Network

In [ ]:
model = ds_boost.build_nn_reg(x_train.shape[1], y_train.shape[1], hidden_layers=2, dropout=0.2, summary=True)

### Train the Neural Network

In [ ]:
model_path = os.path.join("models", "bike_rental.keras")

model = None
model = ds_boost.build_nn_reg(x_train.shape[1], y_train.shape[1], hidden_layers=2, dropout=0.2, summary=True)

callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=1, verbose=0)]

ds_boost.train_nn(
    model,
    x_train,
    y_train,
    validation_data=[x_val, y_val],
    path=model_path,
    epochs=100,
    batch_size=1024,
    callbacks=callbacks,
)

from sklearn.metrics import r2_score

ypred_train = model.predict(x_train)
ypred_val = model.predict(x_val)
print("\nTraining   R2-score: \t{:.3f}".format(r2_score(y_train, ypred_train)))
print("Validation R2-score: \t{:.3f}".format(r2_score(y_val, ypred_val)))

### Evaluate the model

In [ ]:
y_pred_test = model.predict(x_test, verbose=0)
ds_boost.regression_scores(y_test, y_pred_test, return_dataframe=True, index="DNN")

### Make predictions

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))

mean, std = scale_param["cnt"]
predictions = y_pred_test * std + mean
ax.plot(predictions[:, 0], label="Prediction")
ax.plot((test["cnt"] * std + mean).values, label="Data")
ax.set_xlim(right=len(predictions))
ax.legend()

dates = pd.to_datetime(df.iloc[test.index]["dteday"])
dates = dates.apply(lambda d: d.strftime("%b %d"))
ax.set_xticks(np.arange(len(dates))[12::24])
_ = ax.set_xticklabels(dates[12::24], rotation=45)

The model seems quite accurate considering that only two years of data were available.

It fails on the last 10 days of December where we expected more bike riders.

The model was not trained to predict this fall. The training set included data from December 22 to December 31 from one year only (2011), which is not enough. An exploratory analysis and some tests with different models led me to the following results and conclusions:
    
- Adding more features from the dataset has a negligible impact on the accuracy of the model, only increasing the size of the neural network. 

- Removing or replacing the current features makes the model worse.

- The training period December 22 to December 31 in 2011 had more registered riders (mean = 73.6) than the test period in 2012 (mean = 58.3). A ridership drop on Christmas 2012 can be predicted from the weather (worse than 2011), but not the large decline registered. Adding new features could help solve this issue, such as *active registrations* or *Christmas*.

### Compare with classical ML

In [ ]:
# restore training set
x_train = np.vstack([x_train, x_val])
y_train = np.vstack([y_train, y_val])

In [ ]:
ds_boost.ml_regression(x_train, y_train, x_test, y_test)

####  Best tree-based model

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor

lightgbm = LGBMRegressor(
    n_jobs=N_JOBS,
    n_estimators=50,
    max_depth=17,
    num_leaves=50,
    random_state=9,
    force_row_wise=True,
    verbose=-1,
    silent=True,
)
model = MultiOutputRegressor(lightgbm).fit(x_train, y_train)
y_pred = model.predict(x_test)
ds_boost.regression_scores(y_test, y_pred, return_dataframe=True, index="lightGBM")

#### Feature importance

In [ ]:
for i, estimator in enumerate(model.estimators_):
    title = f"Model for target {target[i]}"
    ds_boost.feature_importance(model_features, estimator, title=title)